In [8]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np

In [9]:
parameter_values = pybamm.ParameterValues("Mohtat2020")
parameter_values.update({"SEI kinetic rate constant [m.s-1]": 1e-14})
spm = pybamm.lithium_ion.SPM({"SEI": "ec reaction limited"})

In [10]:
# Calculate stoichiometries at 100% SOC
esoh_model = pybamm.lithium_ion.ElectrodeSOH()
# esoh_sim = pybamm.Simulation(esoh_model, parameter_values=parameter_values)
param = spm.param

Vmin = 3.0
Vmax = 4.2
Cn = parameter_values.evaluate(param.C_n_init)
Cp = parameter_values.evaluate(param.C_p_init)
n_Li_init = parameter_values.evaluate(param.n_Li_particles_init)

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init},
)
print(f"Initial negative electrode SOC: {esoh_sol['x_100'].data[0]:.3f}")
print(f"Initial positive electrode SOC: {esoh_sol['y_100'].data[0]:.3f}")

# Update parameter values with initial conditions
c_n_max = parameter_values.evaluate(param.c_n_max)
c_p_max = parameter_values.evaluate(param.c_p_max)
parameter_values.update(
    {
        "Initial concentration in negative electrode [mol.m-3]": esoh_sol["x_100"].data[0] * c_n_max,
        "Initial concentration in positive electrode [mol.m-3]": esoh_sol["y_100"].data[0] * c_p_max,
    }
)

Initial negative electrode SOC: 0.833
Initial positive electrode SOC: 0.034


In [11]:
Cn

5.973262521249683

In [12]:
Cp

5.795692012196613

In [24]:
sorted(parameter_values.keys())

['1 + dlnf/dlnc',
 'Ambient temperature [K]',
 'Bulk solvent concentration [mol.m-3]',
 'Cation transference number',
 'Cell cooling surface area [m2]',
 'Cell volume [m3]',
 'Current function [A]',
 'EC diffusivity [m2.s-1]',
 'EC initial concentration in electrolyte [mol.m-3]',
 'Edge heat transfer coefficient [W.m-2.K-1]',
 'Electrode height [m]',
 'Electrode width [m]',
 'Electrolyte conductivity [S.m-1]',
 'Electrolyte diffusivity [m2.s-1]',
 'Exchange-current density for plating [A.m-2]',
 'Initial concentration in electrolyte [mol.m-3]',
 'Initial concentration in negative electrode [mol.m-3]',
 'Initial concentration in positive electrode [mol.m-3]',
 'Initial inner SEI thickness [m]',
 'Initial outer SEI thickness [m]',
 'Initial plated lithium concentration [mol.m-3]',
 'Initial temperature [K]',
 'Inner SEI electron conductivity [S.m-1]',
 'Inner SEI lithium interstitial diffusivity [m2.s-1]',
 'Inner SEI open-circuit potential [V]',
 'Inner SEI partial molar volume [m3.mol-

In [25]:
parameter_values.evaluate(param.C_n)

0.07795937097247523

In [21]:
sorted(spm.variables)

['Ambient temperature',
 'Ambient temperature [K]',
 'Average negative particle concentration',
 'Average negative particle concentration [mol.m-3]',
 'Average positive particle concentration',
 'Average positive particle concentration [mol.m-3]',
 'Battery voltage [V]',
 'C-rate',
 'Cell temperature',
 'Cell temperature [K]',
 'Change in measured open circuit voltage',
 'Change in measured open circuit voltage [V]',
 'Current [A]',
 'Current collector current density',
 'Current collector current density [A.m-2]',
 'Current density variable',
 'Discharge capacity [A.h]',
 'EC surface concentration',
 'EC surface concentration [mol.m-3]',
 'Electrode current density',
 'Electrode tortuosity',
 'Electrolyte concentration',
 'Electrolyte concentration [Molar]',
 'Electrolyte concentration [mol.m-3]',
 'Electrolyte current density',
 'Electrolyte current density [A.m-2]',
 'Electrolyte flux',
 'Electrolyte flux [mol.m-2.s-1]',
 'Electrolyte potential',
 'Electrolyte potential [V]',
 'Elec

In [26]:
parameter_values['Lower voltage cut-off [V]']

2.8